# Importing and inspecting the data

We will start by importing the dataset

In [2]:
import pandas as pd
import numpy as np
# Set environment
pd.set_option('max_colwidth', 100)
pd.set_option('precision', 3)
# Read data
df_o=pd.read_csv('./data/train.csv',dtype={'question1':str,'question2':str},index_col='id')

df_o['is_duplicate'].value_counts(normalize=True)*100

0    63.08
1    36.92
Name: is_duplicate, dtype: float64

In [3]:
df_o.shape

(404290, 5)

In [4]:
df_o.dtypes

qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [5]:
df_o.drop_duplicates(inplace=True)
df_o.dropna(how='any',inplace=True)

In [6]:
df_o['question1']=df_o['question1'].astype('str')
df_o['question2']=df_o['question2'].astype('str')

In [93]:
df_o.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0
2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0
3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0
4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0


## Split data

In [56]:
# Split dataset in training and validation dataset, 70/30
# Test dataset provided will not be used until the end, so it can be used for final validation
from sklearn. model_selection import train_test_split
TEST_SIZE = 0.3
X_train, X_val, y_train, y_val = train_test_split(df_o.loc[:,'qid1':'question2'], df_o.loc[:,'is_duplicate'], test_size=TEST_SIZE, random_state=42)
X_train.shape


(283001, 4)

In [58]:
type(y_train)

pandas.core.series.Series

# Generate features

Some ideas to work on:
1. Question lenght in words
2. Shared words
3. Use tf-idf to identify specific words on each pairs
4. Is question clause the same?
5. Syntax similarity: POS tags?
6. Shared synonims (all words or just the key words- verb, object)
7. Similarity of key words: wordnet/synnet similarity score

In [12]:
# Clean up functions
# Remove stopwords
from nltk.corpus import stopwords
from nltk import tokenize

from nltk.corpus import stopwords
import string

def clean_text(s,lower=False):
    """ Tokenize text and remove stop words and punctuation
        s: string to clean
        lower: Boolean if text should be converted to lower case
    """
    
    tok=tokenize.word_tokenize(s)
    if (lower):
        s_nostop=[w.lower() for w in tok if w not in stopwords.words('english')]
    else:
        s_nostop=[w for w in tok if w not in stopwords.words('english')]
    punct=set(string.punctuation)
    s_nopunct=[w for w in s_nostop if w not in punct]
    return s_nopunct
    


#### Feature: different length in words

In [126]:
def n_words(s):
    return(len(s.split()))

In [149]:
X_train['len1']=X_train['question1'].apply(n_words)

In [150]:
X_train['len2']=X_train['question2'].apply(n_words)

In [152]:
X_train['dif_len']=X_train['len1']-X_train['len2']

In [153]:
X_train['dif_len']=X_train['dif_len'].apply(abs)

In [156]:
X_train.drop('len1',1,inplace=True)
X_train.drop('len2',1,inplace=True)
X_train.head()

,qid1,qid2,question1,question2,dif_len
id,,,,,
20128,37998,37999,"How is the working environment at SBI Life, Mumbai?",How stressful is work of SBI clerk?,2
185202,282774,282775,What kinds of questions should I expect on the IBM CAT/IPAT?,What kinds of questions should I expect on the IBM IPAT?,0
107096,176275,176276,"On WhatsApp, it says on the message info that the message has been read with 2 blue ticks and no...",My friend is abroad and I have sent him messages. There is one grey tick next to the messages I ...,15
27940,51849,51850,How do the holy scriptures of Hinduism compare and contrast to those of Taoism?,How do the holy scriptures of Hinduism compare and contrast to those of Italo-Roman paganism?,1
244713,357399,357400,Does beard transplantation really work?,Is beard transplantation worthy?,1


#### Feature: Number of common words

#### Feature: Use tf-idf with cosine similarity
The approach is to identify "vectorize" the strings using tf idf and then find the similarity between the two.
This feature will produce a number between the 2 sentences that will represent how close they are between each other


In [26]:


# Function to determine the tf idf of a list of strings

from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf(l1, l2):
    # 
    result=[]
    for X in zip(l1,l2):
        tfidf_matrix = []
        tfidf_vect=TfidfVectorizer()
        tfidf_sparse=tfidf_vect.fit_transform(X)
        tfidf_matrix=tfidf_sparse.todense()
        #print (tfidf_matrix,"*****")
        cosine_distance=cos_sim(tfidf_matrix)
        result.append(cosine_distance)
    return result

# function to identify the cosine similarity between 2 vectors
import sklearn.metrics.pairwise as metrics
def cos_sim(m):
    cos=metrics.cosine_similarity(m[0],m[1])[0,0]
    return cos

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect=TfidfVectorizer(stop_words='english')
tfidf_list=tfidf_vect.fit_transform(X_train['question1']).toarray()

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [ ]:
q1_list=X_train['question1'].tolist()
q2_list=X_train['question2'].tolist()

In [ ]:
tfidf_cos=tfidf(q1_list[:],q2_list[:])

In [ ]:
type(tfidf_cos)

In [490]:
X_train.ix[:,'tfidf_cos']=pd.Series(tfidf_cos)

In [494]:
X_train

,qid1,qid2,question1,question2,tfidf_cos
id,,,,,
20128,37998,37999,"How is the working environment at SBI Life, Mumbai?",How stressful is work of SBI clerk?,0.143
185202,282774,282775,What kinds of questions should I expect on the IBM CAT/IPAT?,What kinds of questions should I expect on the IBM IPAT?,0.345
107096,176275,176276,"On WhatsApp, it says on the message info that the message has been read with 2 blue ticks and no...",My friend is abroad and I have sent him messages. There is one grey tick next to the messages I ...,0.777
27940,51849,51850,How do the holy scriptures of Hinduism compare and contrast to those of Taoism?,How do the holy scriptures of Hinduism compare and contrast to those of Italo-Roman paganism?,0.078
244713,357399,357400,Does beard transplantation really work?,Is beard transplantation worthy?,0.404
111382,34756,37759,My questions haven't changed. Why are they now being marked as needing improvement?,What should I do if my question is being marked instantly as needing improvement but I don't kno...,0.206
403957,86467,39685,Which phone should I buy under 15k?,Which phone is best to buy under 15k?,NaN
101177,167668,167669,Should I choose MFC Delhi or make another attempt at CAT?,How many attempts did you make and what do you think was your accuracy (approx) in the CAT 14 to...,0.162
78010,133094,133095,Do we really need love?,Do people really need love?,0.372


In [161]:
X_train['tfidf_cos'].fillna(0.0)

KeyError: 'tfidf_cos'

#### Feature: Shared synonyms

#### Feature: Ratio of common nouns (using Jaccard distance)

In [111]:
import nltk
def retrieve_word_pos_nltk(q,pos='N'):
    #tagged=nltk.pos_tag(nltk.word_tokenize(q))
    tagged=nltk.pos_tag(nltk.word_tokenize(q))
    #return [item[0] for item in tagged if item[1][0] == 'N']
    return[tagged]

In [61]:
def jaccard_dis(l1,l2):
    num=len(set(l1).intersection(l2))
    den=len(set(l1).union(l2))
    if den==0:
        return 0
    else:
        return(num/den)

In [121]:
X_train['tok1']=X_train.loc[:,'question1'].apply(nltk.word_tokenize)

In [120]:
X_train['tok2']=X_train.loc[:,'question2'].apply(nltk.word_tokenize)

In [86]:
X_train.to_csv('X_train_nouns.csv')

In [122]:
def jaccard_dis_rows(row):
    l1=row['tok1']
    l2=row['tok2']
    num=len(set(l1).intersection(l2))
    den=len(set(l1).union(l2))
    if den==0:
        return 0
    else:
        return(num/den)

In [123]:
X_train.head()

,qid1,qid2,question1,question2,nouns1,nouns2,ratio_nouns,tok1,tok2
id,,,,,,,,,
20128,37998,37999,"How is the working environment at SBI Life, Mumbai?",How stressful is work of SBI clerk?,"[environment, SBI, Life, Mumbai]","[work, SBI, clerk]",0.167,"[How, is, the, working, environment, at, SBI, Life, ,, Mumbai, ?]","[How, stressful, is, work, of, SBI, clerk, ?]"
185202,282774,282775,What kinds of questions should I expect on the IBM CAT/IPAT?,What kinds of questions should I expect on the IBM IPAT?,"[questions, IBM, CAT/IPAT]","[questions, IBM, IPAT]",0.500,"[What, kinds, of, questions, should, I, expect, on, the, IBM, CAT/IPAT, ?]","[What, kinds, of, questions, should, I, expect, on, the, IBM, IPAT, ?]"
107096,176275,176276,"On WhatsApp, it says on the message info that the message has been read with 2 blue ticks and no...",My friend is abroad and I have sent him messages. There is one grey tick next to the messages I ...,"[WhatsApp, message, info, message, ticks, time, message, grey, checks, time, mean]","[friend, messages, grey, messages, read, ticks, time, read, messages]",0.273,"[On, WhatsApp, ,, it, says, on, the, message, info, that, the, message, has, been, read, with, 2...","[My, friend, is, abroad, and, I, have, sent, him, messages, ., There, is, one, grey, tick, next,..."
27940,51849,51850,How do the holy scriptures of Hinduism compare and contrast to those of Taoism?,How do the holy scriptures of Hinduism compare and contrast to those of Italo-Roman paganism?,"[holy, scriptures, Hinduism, compare, contrast, Taoism]","[holy, scriptures, Hinduism, compare, contrast, Italo-Roman, paganism]",0.625,"[How, do, the, holy, scriptures, of, Hinduism, compare, and, contrast, to, those, of, Taoism, ?]","[How, do, the, holy, scriptures, of, Hinduism, compare, and, contrast, to, those, of, Italo-Roma..."
244713,357399,357400,Does beard transplantation really work?,Is beard transplantation worthy?,"[Does, transplantation]","[transplantation, worthy]",0.333,"[Does, beard, transplantation, really, work, ?]","[Is, beard, transplantation, worthy, ?]"


In [124]:
X_train['jacckard_dis']=X_train.apply(jaccard_dis_rows,axis=1)

In [125]:
X_train.head()

,qid1,qid2,question1,question2,nouns1,nouns2,ratio_nouns,tok1,tok2,jacckard_dis
id,,,,,,,,,,
20128,37998,37999,"How is the working environment at SBI Life, Mumbai?",How stressful is work of SBI clerk?,"[environment, SBI, Life, Mumbai]","[work, SBI, clerk]",0.167,"[How, is, the, working, environment, at, SBI, Life, ,, Mumbai, ?]","[How, stressful, is, work, of, SBI, clerk, ?]",0.267
185202,282774,282775,What kinds of questions should I expect on the IBM CAT/IPAT?,What kinds of questions should I expect on the IBM IPAT?,"[questions, IBM, CAT/IPAT]","[questions, IBM, IPAT]",0.500,"[What, kinds, of, questions, should, I, expect, on, the, IBM, CAT/IPAT, ?]","[What, kinds, of, questions, should, I, expect, on, the, IBM, IPAT, ?]",0.846
107096,176275,176276,"On WhatsApp, it says on the message info that the message has been read with 2 blue ticks and no...",My friend is abroad and I have sent him messages. There is one grey tick next to the messages I ...,"[WhatsApp, message, info, message, ticks, time, message, grey, checks, time, mean]","[friend, messages, grey, messages, read, ticks, time, read, messages]",0.273,"[On, WhatsApp, ,, it, says, on, the, message, info, that, the, message, has, been, read, with, 2...","[My, friend, is, abroad, and, I, have, sent, him, messages, ., There, is, one, grey, tick, next,...",0.286
27940,51849,51850,How do the holy scriptures of Hinduism compare and contrast to those of Taoism?,How do the holy scriptures of Hinduism compare and contrast to those of Italo-Roman paganism?,"[holy, scriptures, Hinduism, compare, contrast, Taoism]","[holy, scriptures, Hinduism, compare, contrast, Italo-Roman, paganism]",0.625,"[How, do, the, holy, scriptures, of, Hinduism, compare, and, contrast, to, those, of, Taoism, ?]","[How, do, the, holy, scriptures, of, Hinduism, compare, and, contrast, to, those, of, Italo-Roma...",0.812
244713,357399,357400,Does beard transplantation really work?,Is beard transplantation worthy?,"[Does, transplantation]","[transplantation, worthy]",0.333,"[Does, beard, transplantation, really, work, ?]","[Is, beard, transplantation, worthy, ?]",0.375


# Applying models

In [37]:
y_train.head()

id
20128     0
185202    1
107096    0
27940     0
244713    1
Name: is_duplicate, dtype: int64

### Baseline algorithm

In [76]:
X_train.loc[:,['ratio_nouns']].head()

,ratio_nouns
id,
20128,0.167
185202,0.500
107096,0.273
27940,0.625
244713,0.333


In [84]:
y_train.to_frame().head()

,is_duplicate
id,
20128,0
185202,1
107096,0
27940,0
244713,1


In [126]:
# We will take LogisticRegression as a simple algorithm to establish a baseline
from sklearn.linear_model import LogisticRegression
import numpy as np

logistic=LogisticRegression()
logistic.fit(X_train.loc[:,['ratio_nouns']],y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [127]:
logistic.score(X_train.loc[:,['ratio_nouns']],y_train)
# Score using Nouns

0.66010014098890113

In [129]:
logistic=LogisticRegression()
logistic.fit(X_train.loc[:,['jacckard_dis']],y_train)
logistic.score(X_train.loc[:,['jacckard_dis']],y_train)
# Score using all words

0.63857371528722517

### Other algorithms

In [18]:
# Flow: 
#   train model with a set of hyperparameters
#   Obtain score and iterate


# Run model on test dataset

# Results

Logistic regression + ratio of common nouns = 0.66


## TEST AREA

In [109]:
df_o[df_o['is_duplicate']==1][['question1','question2']]

,question1,question2
id,,
5,Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?,"I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"
7,How can I be a good geologist?,What should I do to be a great geologist?
11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?
12,What can make Physics easy to learn?,How can you make physics easy to learn?
13,What was your first sexual experience like?,What was your first sexual experience?
15,What would a Trump presidency mean for current international master’s students on an F1 visa?,How will a Trump presidency affect the students presently in US or planning to study in US?
16,What does manipulation mean?,What does manipulation means?
18,Why are so many Quora users posting questions that are readily answered on Google?,Why do people ask Quora questions which can be answered easily by Google?
20,Why do rockets look white?,Why are rockets and boosters painted white?
